In [1]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [160]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [161]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [4]:
file = pd.read_csv("joined_data_v1.csv")

In [5]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
1,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
3,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."


In [6]:
file.drop_duplicates(keep='first', inplace=True)

In [7]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [8]:
overview_df2 = file[["title", "overview"]].copy()
overview_df2.head()

,title,overview
0,The Birth of a Nation,"Nat Turner, a former slave in America, leads a..."
2,Sully,"On 15 January 2009, the world witnessed the 'M..."
4,Deepwater Horizon,A story set on the offshore drilling rig Deepw...
6,Miss Peregrine's Home for Peculiar Children,A teenager finds himself transported to an isl...
8,31,ROB ZOMBIE'S S P O O K H A U S 31


In [9]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
sentenceData = spark.createDataFrame(overview_df2)
sentenceData.show() 

+--------------------+--------------------+
|               title|            overview|
+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|
|               Sully|On 15 January 200...|
|   Deepwater Horizon|A story set on th...|
|Miss Peregrine's ...|A teenager finds ...|
|                  31|ROB ZOMBIE'S  S P...|
|             Snowden|CIA employee Edwa...|
|         Blair Witch|Students on a cam...|
|The Magnificent S...|Seven gun men in ...|
|              Morgan|A corporate risk-...|
|      The Dark Tower|The last Gunsling...|
|          Nine Lives|A stuffy business...|
|       Don't Breathe|A group of teens ...|
|       Suicide Squad|From DC Comics co...|
|Kubo and the Two ...|Kubo mesmerizes t...|
|             Ben-Hur|A falsely accused...|
|            War Dogs|Based on the true...|
|  Hell or High Water|A divorced dad an...|
|          Anthropoid|In December 1941,...|
|       Pete's Dragon|Pete is a mysteri...|
|            Rock Dog|When a rad

In [11]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_b4068e713dfe

In [12]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                      |overview               

In [13]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [14]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------

In [15]:
tokenized.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|
+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|The Magnificent S...|Seven gun men in ...|[seven, gun, men,...|[seven, gun, men,...|
|              Morgan|A corporate risk-...|[a, corpora

# Starting Spacy Similarity code

In [40]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")
import string
punctuations = string.punctuation


In [34]:
overview1 =str(tokenized.select("stopwords").collect()[0])
clean_overview1 = overview1[16:]

overview2 = str(tokenized.select("stopwords").collect()[1])
clean_overview2 = overview2[16:]


In [26]:
clean_overview1

"nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.'])"

In [35]:
clean_overview2

'15\', \'january\', \'2009,\', \'world\', \'witnessed\', "\'miracle", "hudson\'", \'captain\', "\'sully\'", \'sullenberger\', \'glided\', \'disabled\', \'plane\', \'onto\', \'frigid\', \'waters\', \'hudson\', \'river,\', \'saving\', \'lives\', \'155\', \'aboard.\', \'however,\', \'even\', \'sully\', \'heralded\', \'public\', \'media\', \'unprecedented\', \'feat\', \'aviation\', \'skill,\', \'investigation\', \'unfolding\', \'threatened\', \'destroy\', \'reputation\', \'career.\'])'

In [139]:
docA = nlp(clean_overview1)
docB = nlp(clean_overview2)

print(docA.similarity(docB))
type(docA) , type(clean_overview1)

0.984993000906136


(spacy.tokens.doc.Doc, str)

In [123]:
print(docB)

15', 'january', '2009,', 'world', 'witnessed', "'miracle", "hudson'", 'captain', "'sully'", 'sullenberger', 'glided', 'disabled', 'plane', 'onto', 'frigid', 'waters', 'hudson', 'river,', 'saving', 'lives', '155', 'aboard.', 'however,', 'even', 'sully', 'heralded', 'public', 'media', 'unprecedented', 'feat', 'aviation', 'skill,', 'investigation', 'unfolding', 'threatened', 'destroy', 'reputation', 'career.'])


In [104]:
overviewC = str(tokenized.select("stopwords").collect()[2])
clean_overviewC = overviewC[16:]
docC = nlp(clean_overviewC)

In [106]:
clean_overviewC

"story', 'set', 'offshore', 'drilling', 'rig', 'deepwater', 'horizon,', 'exploded', 'april', '2010', 'created', 'worst', 'oil', 'spill', 'u.s.', 'history.'])"

In [105]:
print(docA.similarity(docC))

0.9805625913523139


In [107]:
print(docB.similarity(docC))

0.9873666383211166


## adding columns for similarities

In [114]:
from pyspark.sql.functions import lit

In [178]:
ll = []
for f in tokenized.collect():

    overview_row = str(f.stopwords)
    #print(overview_row)
    doc_row = nlp(overview_row)

    compare = docA.similarity(doc_row)
    ll.append(compare)
# sample = tokenized.withColumn('similarity', lit(str(ll)))
# ll   


In [179]:
ll

[0.9995973840399037,
 0.9849470144479944,
 0.9805784207416413,
 0.9815930097071623,
 0.9608167130469573,
 0.9767509005577233,
 0.9741712529231993,
 0.9852484058711272,
 0.9723964056257384,
 0.9870985405152113,
 0.9673086756812984,
 0.9805259498839797,
 0.9876107808980823,
 0.9812733500570661,
 0.9814675819225775,
 0.9876015748717686,
 0.9789823192858077,
 0.9833466595878102,
 0.9860609180028044,
 0.9851739823392089,
 0.9850632975579839,
 0.9867049451864595,
 0.9792555593068961,
 0.9727223836143322,
 0.9820477463838587,
 0.9778408330259459,
 0.9850906470458799,
 0.9863823968845286,
 0.9839772552171261,
 0.9840892288397363,
 0.9869574413309489,
 0.9855907789143409,
 0.9801666476245171,
 0.985536054537955,
 0.9844556054582799,
 0.9863013942989776,
 0.9822162471977279,
 0.9849123078353049,
 0.9878330486722571,
 0.9910508146761611,
 0.9812905813495025,
 0.9843608002215642,
 0.988188436083859,
 0.9840372845234439,
 0.9847398108239271,
 0.9826507514560081,
 0.9842093881976233,
 0.984585541277

In [159]:
# from pyspark.sql import SQLContext

In [163]:
from pyspark.sql.functions import monotonically_increasing_id

In [182]:
from pyspark.sql.types import *
cSchema = StructType([StructField("Similarity", ArrayType(FloatType()))])
b= spark.createDataFrame(ll, FloatType()).toDF("x")
#b = spark.createDataFrame(ll, schema = cSchema)

tokenized = tokenized.withColumn("row_idx", monotonically_increasing_id())
b = b.withColumn("row_idx", monotonically_increasing_id())

final_df = tokenized.join(b, tokenized.row_idx == b.row_idx).\
            drop("row_idx")
final_df.show()

TypeError: field value: FloatType can not accept object 0.9995973840399037 in type <class 'numpy.float64'>

In [152]:
sample.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|          similarity|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|[0.99959738403990...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|[0.99959738403990...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|[0.99959738403990...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|[0.99959738403990...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|[0.99959738403990...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|[0.99959738403990...|
|         Blair Witch|Students on a c

## Removed punctuation

In [133]:
docI = nlp(clean_overview1)
tokenI = [tok.lemma_.lower() for tok in docI]
tokenI = [tok for tok in tokenI if tok not in punctuations]
# tokenI = ' '.join(tokenI)

print(tokenI)

['nat', 'turner', 'former', 'slave', 'america', 'lead', 'liberation', 'movement', '1831', 'free', 'african', 'american', 'virgina', 'result', 'violent', 'retaliation', 'white']


In [132]:
docII = nlp(clean_overview2)
tokenII = [tok.lemma_.lower().strip() for tok in docII]
tokenII = [tok for tok in tokenII if tok not in punctuations]

print(tokenII)

['15', 'january', '2009', 'world', 'witness', 'miracle', 'hudson', 'captain', 'sully', 'sullenberger', 'glide', 'disabled', 'plane', 'onto', 'frigid', 'water', 'hudson', 'river', 'saving', 'life', '155', 'aboard', 'however', 'even', 'sully', 'herald', 'public', 'medium', 'unprecedented', 'feat', 'aviation', 'skill', 'investigation', 'unfold', 'threaten', 'destroy', 'reputation', 'career']


In [134]:
for f in tokenized.collect():
    doc_r=''
    overview_r = str(f.stopwords)
    print(overview_r)
    doc_r = nlp(overview_r)
    comp = docI.similarity(doc_r)
    samp = tokenized.withColumn('similarity', lit(str(compare)))

['nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.']
['15', 'january', '2009,', 'world', 'witnessed', "'miracle", "hudson'", 'captain', "'sully'", 'sullenberger', 'glided', 'disabled', 'plane', 'onto', 'frigid', 'waters', 'hudson', 'river,', 'saving', 'lives', '155', 'aboard.', 'however,', 'even', 'sully', 'heralded', 'public', 'media', 'unprecedented', 'feat', 'aviation', 'skill,', 'investigation', 'unfolding', 'threatened', 'destroy', 'reputation', 'career.']
['story', 'set', 'offshore', 'drilling', 'rig', 'deepwater', 'horizon,', 'exploded', 'april', '2010', 'created', 'worst', 'oil', 'spill', 'u.s.', 'history.']
['teenager', 'finds', 'transported', 'island', 'must', 'help', 'protect', 'group', 'orphans', 'special', 'powers', 'creatures', 'intent', 'destroying', 'them.']
['rob', "zombie's", '', 'p', 'o', 'o', 'k', '', 'h', 'u', '31']
['cia', 'employee', 'ed

['two', 'years', 'choosing', 'kill', 'man', 'killed', 'son,', 'former', 'police', 'sergeant', 'leo', 'barnes', 'become', 'head', 'security', 'senator', 'charlene', 'roan,', 'front', 'runner', 'next', 'presidential', 'election', 'due', 'vow', 'eliminate', 'purge.', 'night', 'final', 'purge,', 'betrayal', 'within', 'government', 'forces', 'barnes', 'roan', 'onto', 'street', 'must', 'fight', 'survive', 'night.']
['1863,', 'mississippi', 'farmer', 'newt', 'knight', 'serves', 'medic', 'confederate', 'army.', 'opposed', 'slavery,', 'knight', 'rather', 'help', 'wounded', 'fight', 'union.', 'nephew', 'dies', 'battle,', 'newt', 'returns', 'home', 'jones', 'county', 'safeguard', 'family', 'soon', 'branded', 'outlaw', 'deserter.', 'forced', 'flee,', 'finds', 'refuge', 'group', 'runaway', 'slaves', 'hiding', 'swamps.', 'forging', 'alliance', 'slaves', 'farmers,', 'knight', 'leads', 'rebellion', 'forever', 'change', 'history.']
['injured', 'surfer', 'stranded', 'buoy', 'needs', 'get', 'back', 'shor

['lorraine', 'ed', 'warren', 'travel', 'north', 'london', 'help', 'single', 'mother', 'raising', 'four', 'children', 'alone', 'house', 'plagued', 'malicious', 'spirits.']
['devoted', 'lifeguard', 'mitch', 'buchannon', 'butts', 'heads', 'brash', 'new', 'recruit.', 'together,', 'uncover', 'local', 'criminal', 'plot', 'threatens', 'future', 'bay.']
['island', 'populated', 'entirely', 'happy,', 'flightless', 'birds', 'almost', 'entirely.', 'paradise,', 'red,', 'bird', 'temper', 'problem,', 'speedy', 'chuck,', 'volatile', 'bomb', 'always', 'outsiders.', 'island', 'visited', 'mysterious', 'green', 'piggies,', 'it’s', 'unlikely', 'outcasts', 'figure', 'pigs', 'to.']
['story', 'young', 'man', 'arrives', 'hollywood', '1930s', 'hoping', 'work', 'film', 'industry,', 'falls', 'love,', 'finds', 'swept', 'vibrant', 'café', 'society', 'defined', 'spirit', 'age.']
['titan', 'industry', 'sent', 'prison', 'caught', 'insider', 'trading.', 'emerges', 'ready', 'rebrand', "america's", 'latest', 'sweetheart,

['samantha', 'kingston', 'everything.', 'then,', 'everything', 'changes.', 'one', 'fateful', 'night,', 'wakes', 'future', 'all.', 'trapped', 'reliving', 'day', 'over,', 'begins', 'question', 'perfect', 'life', 'really', 'was.']
['london', 'prime', "minister's", 'funeral,', 'mike', 'banning', 'discovers', 'plot', 'assassinate', 'attending', 'world', 'leaders.']
['modern', 'love', 'story', 'set', 'austin,', 'texas', 'music', 'scene,', 'two', 'entangled', 'couples', '—', 'struggling', 'songwriters', 'faye', 'bv,', 'music', 'mogul', 'cook', 'waitress', 'ensnares', '—', 'chase', 'success', 'rock', '‘n’', 'roll', 'landscape', 'seduction', 'betrayal.']
['faith', 'based', 'movie.', 'young', 'girl', 'suffering', 'rare', 'digestive', 'disorder', 'finds', 'miraculously', 'cured', 'surviving', 'terrible', 'accident.', 'based', 'book', "'three", 'miracles', "heaven'", 'christy', 'beam.']
['live-action', 'adaptation', "disney's", 'version', 'classic', "'beauty", "beast'", 'tale', 'cursed', 'prince',

['ambitious', 'young', 'executive', 'sent', 'retrieve', "company's", 'ceo', 'idyllic', 'mysterious', '"wellness', 'center"', 'remote', 'location', 'swiss', 'alps', 'soon', 'suspects', "spa's", 'miraculous', 'treatments', 'seem.']
['determined', 'prove', 'herself,', 'officer', 'judy', 'hopps,', 'first', 'bunny', "zootopia's", 'police', 'force,', 'jumps', 'chance', 'crack', 'first', 'case', '-', 'even', 'means', 'partnering', 'scam-artist', 'fox', 'nick', 'wilde', 'solve', 'mystery.']
['julia', 'becomes', 'worried', 'boyfriend,', 'holt', 'explores', 'dark', 'urban', 'legend', 'mysterious', 'videotape', 'said', 'kill', 'watcher', 'seven', 'days', 'viewing.', 'sacrifices', 'save', 'boyfriend', 'makes', 'horrifying', 'discovery:', '"movie', 'within', 'movie"', 'one', 'ever', 'seen', 'before.']
['retreating', 'life', 'tragedy,', 'man', 'questions', 'universe', 'writing', 'love,', 'time', 'death.', 'receiving', 'unexpected', 'answers,', 'begins', 'see', 'things', 'interlock', 'even', 'loss', 

['taking', 'place', 'alien', 'crafts', 'land', 'around', 'world,', 'expert', 'linguist', 'recruited', 'military', 'determine', 'whether', 'come', 'peace', 'threat.']
['boy', 'imagines', 'monster', 'helps', 'deal', 'difficult', 'life', 'see', 'world', 'different', 'way.']
['wwii', 'american', 'army', 'medic', 'desmond', 't.', 'doss,', 'served', 'battle', 'okinawa,', 'refuses', 'kill', 'people', 'becomes', 'first', 'conscientious', 'objector', 'american', 'history', 'receive', 'congressional', 'medal', 'honor.']
['rachel', 'watson,', 'devastated', 'recent', 'divorce,', 'spends', 'daily', 'commute', 'fantasizing', 'seemingly', 'perfect', 'couple', 'live', 'house', 'train', 'passes', 'every', 'day,', 'one', 'morning', 'sees', 'something', 'shocking', 'happen', 'becomes', 'entangled', 'mystery', 'unfolds.']
['career', 'destroyed,', 'brilliant', 'arrogant', 'surgeon', 'gets', 'new', 'lease', 'life', 'sorcerer', 'takes', 'wing', 'trains', 'defend', 'world', 'evil.']
['tender,', 'heartbreaking

In [148]:
samp.show()

+--------------------+--------------------+--------------------+--------------------+-----------------+
|               title|            overview|               words|           stopwords|       similarity|
+--------------------+--------------------+--------------------+--------------------+-----------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|0.985549637580337|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|0.985549637580337|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|0.985549637580337|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|0.985549637580337|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|0.985549637580337|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|0.985549637580337|
|         Blair Witch|Students on a cam...|[students, on, a,...|

In [103]:
sim = docI.similarity(docII)
sim

0.984993000906136

In [72]:
sim1 = overview1.similarity(overview2)
sim1

AttributeError: 'str' object has no attribute 'similarity'

In [98]:
text_compareX = []
docx = nlp(clean_overview1, disable=['parser'])

tokensx = [tok.lemma_ for tok in docx if tok not in punctuations]
tokensx = ' '.join(tokensx)
text_compareX.append(tokensx)

text_compareX

TypeError: 'in <string>' requires string as left operand, not spacy.tokens.token.Token

In [93]:
text_compare1 = []
doc1 = nlp(clean_overview1, disable=['parser'])

tokens1 = [tok.lemma_.lower().strip() for tok in doc1]
tokens1 = [tok for tok in tokens1 if tok not in punctuations]
tokens1 = ' '.join(tokens1)
text_compare1.append(tokens1)

text_compare1

['nat turner former slave america lead liberation movement 1831 free african american virgina result violent retaliation white']

In [99]:
simil = tokens1.similarity(tokens2)
simil

AttributeError: 'str' object has no attribute 'similarity'

In [94]:
text_compare2 = []
doc2 = nlp(clean_overview2, disable=['parser'])

tokens2 = [tok.lemma_.lower().strip() for tok in doc2]
tokens2 = [tok for tok in tokens2 if tok not in punctuations]
tokens2 = ' '.join(tokens2)
text_compare2.append(tokens2)

text_compare2

['15 january 2009 world witness miracle hudson captain sully sullenberger glide disabled plane onto frigid water hudson river saving life 155 aboard however even sully herald public medium unprecedented feat aviation skill investigation unfold threaten destroy reputation career']

In [95]:
comp = text_compare1.similarity(text_compare2)
comp

AttributeError: 'list' object has no attribute 'similarity'

In [55]:
length = len(text_compare)
i = 0

while i < length: 
    for t[0+i] in text_compare:
        for t[1+i] in text_compare:
            print(t[0+i].text, t[1+i].text, t[0+i].similarity(t[1+i]))
    

NameError: name 't' is not defined

In [59]:
for token in tokens1:
    for token in tokens2:
        print(tokens1, tokens2, tokens1.similarity(tokens2))

AttributeError: 'str' object has no attribute 'similarity'